# Esercizio 5 - Topic Modelling

In [10]:
import wikipediaapi

## Creo 4 documenti di input

In [12]:
# Try wiki api
wiki_wiki = wikipediaapi.Wikipedia('en')

lebanon_page = wiki_wiki.page('Lebanon')
bike_page = wiki_wiki.page('Racing bicycle')
labrador_page = wiki_wiki.page('Labrador retriever')
indie_page = wiki_wiki.page('Indie rock')

lebanon_text = lebanon_page.text
bike_text = bike_page.text
labrador_text = labrador_page.text
indie_text = indie_page.text

# Create documents as txt files
with open('../data/docs/lebanon.txt', 'w') as f:
    f.write(lebanon_text)

with open('../data/docs/bike.txt', 'w') as f:
    f.write(bike_text)

with open('../data/docs/labrador.txt', 'w') as f:
    f.write(labrador_text)

with open('../data/docs/indie.txt', 'w') as f:
    f.write(indie_text)
